In [283]:
import sys
import pandas as pd
import re
import numpy as np
import preprocessor as p
import nltk
from nltk.tokenize import WhitespaceTokenizer
import urllib.request, json 

def clean_tweets(df):
    tempArr = []
    for line in df:
        # send to tweet_processor
        tmpL = p.clean(line)
        # remove everything except letters and whitespaces
        tmpL = re.sub(r'[^\w\s]|[\d]', ' ', tmpL)
        # convert to lowercase
        tmpL = tmpL.lower()
        tempArr.append(tmpL)
    return tempArr

REPLACE_NO_SPACE = re.compile("(\d)|(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

train = pd.read_csv(r"C:\Users\Admin\OneDrive\Documents\Webdev\hatespeech\train.csv", nrows=5000)
#train = pd.read_csv(r"C:\Users\jmest\Documents\Files\C4S2\Thesis\ThesisFiles-main\ThesisFiles-main\hatespeech\train.csv", nrows=5000)


train_tweet = clean_tweets(train["text"])
train_tweet = pd.DataFrame(train_tweet)
train["clean_tweet"] = train_tweet
sum(train['clean_tweet'] == '')
train['clean_tweet'] = train['clean_tweet'].replace('', np.NaN)
train.dropna(axis='rows')



,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about...
...,...,...,...
4994,@jefk_rew pity those poor people who were invo...,1,pity those poor people who were involved in th...
4995,Awww DUTERTE Na wag Lang si Roxas,0,awww duterte na wag lang si roxas
4996,RT @mikkieugenio: If the SC disqualifies Poe a...,0,if the sc disqualifies poe as president eit...
4997,"""Pag naging presidente si Binay, wala kayong t...",0,pag naging presidente si binay wala kayong t...


In [284]:
train['clean_tweet'] = train['clean_tweet'].replace('', float('NaN'), regex = True)
train.dropna(inplace= True)
train = train.reset_index(drop=True)
first_column = train.pop('label')
train.insert(0,'label',first_column)
train.dropna(axis='rows')

with urllib.request.urlopen("https://raw.githubusercontent.com/stopwords-iso/stopwords-tl/master/stopwords-tl.json") as url:
    stopwords = json.loads(url.read().decode())
    print(stopwords)


train['rm_stpwrds'] = train['clean_tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords) and not word.isdigit()]))
train['tokenize'] = train['rm_stpwrds'].apply(nltk.tokenize.WhitespaceTokenizer().tokenize)

['akin', 'aking', 'ako', 'alin', 'am', 'amin', 'aming', 'ang', 'ano', 'anumang', 'apat', 'at', 'atin', 'ating', 'ay', 'bababa', 'bago', 'bakit', 'bawat', 'bilang', 'dahil', 'dalawa', 'dapat', 'din', 'dito', 'doon', 'gagawin', 'gayunman', 'ginagawa', 'ginawa', 'ginawang', 'gumawa', 'gusto', 'habang', 'hanggang', 'hindi', 'huwag', 'iba', 'ibaba', 'ibabaw', 'ibig', 'ikaw', 'ilagay', 'ilalim', 'ilan', 'inyong', 'isa', 'isang', 'itaas', 'ito', 'iyo', 'iyon', 'iyong', 'ka', 'kahit', 'kailangan', 'kailanman', 'kami', 'kanila', 'kanilang', 'kanino', 'kanya', 'kanyang', 'kapag', 'kapwa', 'karamihan', 'katiyakan', 'katulad', 'kaya', 'kaysa', 'ko', 'kong', 'kulang', 'kumuha', 'kung', 'laban', 'lahat', 'lamang', 'likod', 'lima', 'maaari', 'maaaring', 'maging', 'mahusay', 'makita', 'marami', 'marapat', 'masyado', 'may', 'mayroon', 'mga', 'minsan', 'mismo', 'mula', 'muli', 'na', 'nabanggit', 'naging', 'nagkaroon', 'nais', 'nakita', 'namin', 'napaka', 'narito', 'nasaan', 'ng', 'ngayon', 'ni', 'nila',

In [285]:
train[['text','clean_tweet','rm_stpwrds']].head(10)

,text,clean_tweet,rm_stpwrds
0,Inaasahan na ni Vice President Jejomar Binay n...,inaasahan na ni vice president jejomar binay n...,inaasahan vice president jejomar binay taong
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,mar roxas tang ina tuwid na daan daw eh sya...,mar roxas tang ina tuwid daan daw eh sya nga d...
2,Salamat sa walang sawang suporta ng mga taga m...,salamat sa walang sawang suporta ng mga taga m...,salamat sawang suporta taga makati pagbabalik ...
3,@rapplerdotcom putangina mo binay TAKBO PA,putangina mo binay takbo pa,putangina mo binay takbo
4,"Binay with selective amnesia, forgetting about...",binay with selective amnesia forgetting about...,binay with selective amnesia forgetting about ...
5,It doesn't matter whoever won between Duterte ...,it doesn t matter whoever won between duterte ...,it doesn t matter whoever won between duterte ...
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,nognog pero nognog din ang nag malasakit wtf...,nognog nognog nag malasakit wtf tangina mo bin...
7,#OnlyB1nay ?? #FB https://t.co/QEQnsK67Gm,,
8,What Abi Binay said on running for Makati mayo...,what abi binay said on running for makati mayor,what abi binay said on running for makati mayor
9,Srsly. How can Binay do away with no tax for t...,srsly how can binay do away with no tax for t...,srsly how can binay do away with no tax for th...


In [286]:
print(train.shape)

(4756, 5)


In [287]:
VOWELS = "aeiouAEIOU"
CONSONANTS = "bcdfghklmnngpqrstvwyBCDFGHKLMNNGPQRSTVWY"

""" 
	Affixes
"""
PREFIX_SET = [
	'nakikipag', 'pakikipag',
	'pinakama', 'pagpapa',
	'pinagka', 'panganga', 
	'makapag', 'nakapag', 
	'tagapag', 'makipag', 
	'nakipag', 'tigapag',
	'pakiki', 'magpa',
	'napaka', 'pinaka',
	'ipinag', 'pagka', 
	'pinag', 'mapag', 
	'mapa', 'taga', 
	'ipag', 'tiga', 
	'pala', 'pina', 
	'pang', 'naka',
	'nang', 'mang',
	'sing',
	'ipa', 'pam',
	'pan', 'pag',
	'tag', 'mai',
	'mag', 'nam',
	'nag', 'man',
	'may', 'ma',
	'na', 'ni',
	'pa', 'ka',
	'um', 'in',
	'i',
]

INFIX_SET = [
	'um', 'in',
]

SUFFIX_SET = [
	'syon','dor', 
	'ita', 'han', 
	'hin', 'ing', 
	'ang', 'ng', 
	'an', 'in', 
	'g',
]

PERIOD_FLAG = True
PASS_FLAG = False

In [288]:

def check_vowel(substring):
	"""
		Checks if the substring is a vowel.
			letters: substring to be tested
		returns BOOLEAN
	"""

	return all(letter in VOWELS for letter in substring)


def check_consonant(substring):
	"""
		Checks if the letter is a consonant.
			letter: substring to be tested
		returns BOOLEAN
	"""

	return all(letter in CONSONANTS for letter in substring)

In [289]:
def change_letter(token, index, letter):
	"""
		Replaces a letter in a token.
			token: word to be used
			index: index of the letter
			letter: letter used to replace
		returns STRING
	"""
	
	_list = list(token)
	_list[index] = letter

	return ''.join(_list)

In [290]:
def count_vowel(token):
	"""
		Count vowels in a given token.
			token: string to be counted for vowels
		returns INTEGER
	"""

	count = 0

	for tok in token:
		if check_vowel(tok):
			count+=1

	return count


def count_consonant(token):
	"""
		Count consonants in a given token.
			token: string to be counted for consonants
		returns INTEGER
	"""

	count = 0

	for tok in token:
		if check_consonant(tok):
			count+=1

	return count

In [291]:
def check_validation(token):
    with open('validation.txt', 'r') as valid:
        data = valid.read().replace('\n', ' ').split(' ')

    return token in data


In [292]:
def clean_repitition(token, REPITITION):
	"""
		Checks token for repitition. (ex. nakakabaliw = nabaliw)
			token: word to be stemmed repitition
		returns STRING
	"""

	if check_validation(token):
		return token

	if len(token) >= 4:
		if check_vowel(token[0]):
			if token[0] == token[1]:
				REPITITION.append(token[0])
				return token[1:]

		elif check_consonant(token[0]) and count_vowel(token) >= 2:
			if token[0: 2] == token[2: 4] and len(token) - 2 >= 4:
				REPITITION.append(token[2:4])
				return token[2:]
			
			elif token[0: 3] == token[3: 6] and len(token) - 3 >= 4:
				REPITITION.append(token[3:6])
				return token[3:]

	return token

In [293]:
def clean_suffix(token, SUFFIX):
    """
    Checks token for suffixes. (ex. bigayan = bigay)
        token: word to be stemmed for suffixes
    returns STRING
    """

    SUF_CANDIDATE = []

    if check_validation(token):
        return token

    for suffix in SUFFIX_SET:
        if len(token) - len(suffix) >= 3 and count_vowel(token[0:len(token) - len(suffix)]) >= 2 and count_consonant(token[0:len(token) - len(suffix)]) >= 1:
            if token[len(token) - len(suffix): len(token)] == suffix:
                if len(suffix) == 2 and not count_consonant(token[0:len(token) - len(suffix)]) >= 1:
                    continue

                if count_vowel(token[0: len(token) - len(suffix)]) >= 2:
                    if suffix == 'ang' and check_consonant(token[-4]) \
                            and token[-4] != 'r' and token[-5] != 'u':
                        continue

                    print(token[0: len(token) - len(suffix)] + " : " + suffix)

                    if check_validation(token[0: len(token) - len(suffix)]):
                        SUFFIX.append(suffix)
                        return token[0: len(token) - len(suffix)] + 'a' if suffix == 'ita' \
                            else token[0: len(token) - len(suffix)]

                    elif len(SUF_CANDIDATE) == 0:
                        SUF_CANDIDATE.append(suffix)
                        SUF_CANDIDATE.append(token[0: len(token) - len(suffix)])

    if (len(SUF_CANDIDATE) == 2):
        SUFFIX = SUF_CANDIDATE[0]
        return SUF_CANDIDATE[1][0: len(token) - len(SUFFIX)] + 'a' if SUFFIX == 'ita' \
            else SUF_CANDIDATE[1][0: len(token) - len(SUFFIX)]

    return token


In [294]:
def clean_infix(token, INFIX):
	"""
		Checks token for infixes. (ex. bumalik = balik)
			token: word to be stemmed for infixes
		returns STRING
	"""

	if check_validation(token):
		return token

	for infix in INFIX_SET:
		if len(token) - len(infix) >= 3 and count_vowel(token[len(infix):]) >= 2:
			if token[0] == token[4] and token[1: 4] == infix:
				INFIX.append(infix)
				return token[4:]

			elif token[2] == token[4] and token[1: 3] == infix:
				INFIX.append(infix)
				return token[0] + token[3:]

			elif token[1: 3] == infix and check_vowel(token[3]):
				INFIX.append(infix)
				return token[0] + token[3:]

	return token

In [295]:
def clean_prefix(token,	 PREFIX):
	"""
		Checks token for prefixes. (ex. naligo = ligo)
			token: word to be stemmed for prefixes
		returns STRING
	"""

	if check_validation(token):
		return token

	for prefix in PREFIX_SET:
		if len(token) - len(prefix) >= 3 and \
			count_vowel(token[len(prefix):]) >= 2:

			if prefix == ('i') and check_consonant(token[2]):
				continue

			if '-' in token:	
				token = token.split('-')

				if token[0] == prefix and check_vowel(token[1][0]):
					PREFIX.append(prefix)
					return token[1]

				token = '-'.join(token)

			if token[0: len(prefix)] == prefix:
				if count_vowel(token[len(prefix):]) >= 2:
					# if check_vowel(token[len(token) - len(prefix) - 1]):
				# 	continue

					if prefix == 'panganga':
						PREFIX.append(prefix)
						return 'ka' + token[len(prefix):]
					
					PREFIX.append(prefix)
					return token[len(prefix):]

	return token

In [296]:
def clean_duplication(token, DUPLICATE):
	"""
		Checks token for duplication. (ex. araw-araw = araw)
			token: word to be stemmed duplication
		returns STRING
	"""

	if check_validation(token):
		return token

	if '-' in token and token.index('-') != 0 and \
		token.index('-') != len(token) -  1:

		split = token.split('-')

		if all(len(tok) >= 3 for tok in split):
			if split[0] == token[1] or split[0][-1] == 'u' and change_letter(split[0], -1, 'o') == split[1] or \
				split[0][-2] == 'u' and change_letter(split[0], -2, 'o')  == split[1]:
				DUPLICATE.append(split[0])
				return split[0]

			elif split[0] == split[1][0:len(split[0])]:
				DUPLICATE.append(split[1])
				return split[1]

			elif split[0][-2:] == 'ng':
				if split[0][-3] == 'u':
					if split[0][0:-3] + 'o' == split[1]:
						DUPLICATE.append(split[1])
						return split[1]

				if split[0][0:-2] == split[1]:
					DUPLICATE.append(split[1])
					return split[1]

		else:
			return '-'.join(split)
	
	return token

In [297]:
def clean_repitition(token, REPITITION):
	"""
		Checks token for repitition. (ex. nakakabaliw = nabaliw)
			token: word to be stemmed repitition
		returns STRING
	"""

	if check_validation(token):
		return token

	if len(token) >= 4:
		if check_vowel(token[0]):
			if token[0] == token[1]:
				REPITITION.append(token[0])
				return token[1:]

		elif check_consonant(token[0]) and count_vowel(token) >= 2:
			if token[0: 2] == token[2: 4] and len(token) - 2 >= 4:
				REPITITION.append(token[2:4])
				return token[2:]
			
			elif token[0: 3] == token[3: 6] and len(token) - 3 >= 4:
				REPITITION.append(token[3:6])
				return token[3:]

	return token


In [298]:

def clean_stemmed(token, CLEANERS, REPITITION):
	"""
		Checks for left-over affixes and letters.
			token: word to be cleaned for excess affixes/letters
		returns STRING
	"""

	global PERIOD_FLAG
	global PASS_FLAG

	CC_EXP = ['dr', 'gl', 'gr', 'ng', 'kr', 'kl', 'kw', 'ts', 'tr', 'pr', 'pl', 'pw', 'sw', 'sy'] # Consonant + Consonant Exceptions

	if token[-1] == '.' and PASS_FLAG == False:
		PERIOD_FLAG = True

	if not check_vowel(token[-1]) and not check_consonant(token[-1]):
		CLEANERS.append(token[-1])
		token = token[0:-1]

	if not check_vowel(token[0]) and not check_consonant(token[0]):
		CLEANERS.append(token[0])
		token = token[1:]

	if check_validation(token):
		return token

	if len(token) >= 3 and count_vowel(token) >= 2:
		token = clean_repitition(token,	REPITITION)

		if check_consonant(token[-1]) and token[- 2] == 'u':
			CLEANERS.append('u')
			token = change_letter(token, -2, 'o')

		if token[len(token) - 1] == 'u':
			CLEANERS.append('u')
			token = change_letter(token, -1, 'o')

		if token[-1] == 'r':
			CLEANERS.append('r')
			token = change_letter(token, -1, 'd')

		if token[-1] == 'h' and check_vowel(token[-1]):
			CLEANERS.append('h')
			token = token[0:-1]

		# if token[0] == 'i':
		# 	token = token[1:]

		if token[0] == token[1]:
			CLEANERS.append(token[0])
			token = token[1:]

		if (token[0: 2] == 'ka' or token[0: 2] == 'pa') and check_consonant(token[2]) \
			and count_vowel(token) >= 3:
			
			CLEANERS.append(token[0: 2])
			token = token[2:]

		if(token[-3:]) == 'han' and count_vowel(token[0:-3]) == 1:
			CLEANERS.append('han')
			token = token[0:-3] + 'i'

		if(token[-3:]) == 'han' and count_vowel(token[0:-3]) > 1:
			CLEANERS.append('han')
			token = token[0:-3]

		if len(token) >= 2 and count_vowel(token) >= 3:
			if token[-1] == 'h' and check_vowel(token[-2]):
				CLEANERS.append('h')
				token = token[0:-1]

		if len(token) >= 6 and token[0:2] == token[2:4]:
			CLEANERS.append('0:2')
			token = token[2:]

		if any(REP[0] == 'r' for REP in REPITITION):
			CLEANERS.append('r')
			token = change_letter(token, 0, 'd')

		if token[-2:] == 'ng' and token[-3] == 'u':
			CLEANERS.append('u')
			token = change_letter(token, -3, 'o')

		if token[-1] == 'h':
			CLEANERS.append('h')
			token = token[0:-1]

		if any(token[0:2] != CC for CC in CC_EXP) and check_consonant(token[0:2]):
			CLEANERS.append(token[0:2])
			token = token[1:]

	return token


In [299]:
def stemmer(tokens):

	global PERIOD_FLAG
	global PASS_FLAG

	pre_stem     = inf_stem = suf_stem = rep_stem = \
		du1_stem = du2_stem = cle_stem = '-'
	word_info    = {}
	PREFIX     = []
	INFIX      = []
	SUFFIX     = []
	DUPLICATE  = []
	REPITITION = []
	CLEANERS   = []

	for token in tokens:
			
			word_info["word"] = token
			
			if (PERIOD_FLAG == True and token[0].isupper()) or \
				(PERIOD_FLAG == False and token[0].islower()):

				token 	 = token.lower()		
				du1_stem = clean_duplication(token, DUPLICATE)
				pre_stem = clean_prefix(du1_stem, PREFIX)
				rep_stem = clean_repitition(pre_stem, REPITITION)
				inf_stem = clean_infix(rep_stem, INFIX)
				rep_stem = clean_repitition(inf_stem, REPITITION)
				suf_stem = clean_suffix(rep_stem, SUFFIX)
				du2_stem = clean_duplication(suf_stem, DUPLICATE)
				cle_stem = clean_stemmed(du2_stem, CLEANERS, REPITITION)
				cle_stem = clean_duplication(cle_stem, DUPLICATE)

			else:
				PERIOD_FLAG = False
				cle_stem = clean_stemmed(token, CLEANERS, REPITITION)
				word_info["root"]   = token
				word_info["prefix"] = '[]'
				word_info["infix"]  = '[]'
				word_info["suffix"] = '[]'
				word_info["repeat"] = '[]'
				word_info["dupli"]  = '[]'
				word_info["clean"]   = '[]'
			
		


In [300]:
train['tokenize'].to_csv('tokenize.txt', sep=' ', index=False)


In [301]:
# Perform stemming
train['stemmed'] = train['tokenize'].apply(stemmer)


tao : ng
sawa : ng
forgett : ing
forgetti : ng
forgettin : g
prepar : ing
prepari : ng
preparin : g
nogno : g
nogno : g
runni : ng
runnin : g
earn : ing
earni : ng
earnin : g
hirap : an
endi : ng
endin : g
aga : in
nogno : g
break : ing
breaki : ng
breakin : g
ito : ng
niyo : ng
batikus : in
talaga : ng
para : ng
gusto : ng
hirapa : ng
para : ng
kipa : g
talun : in
bei : ng
bein : g
para : ng
uili : an
kair : ita
goberna : dor
salamat : an
nakaw : an
handa : ng
ibago : ng
nogno : g
stumpi : ng
stumpin : g
talaga : ng
calli : ng
callin : g
bayar : an
tanung : in
inta : ng
intan : g
basa : hin
endors : ing
endorsi : ng
endorsin : g
tulung : an
ombudsm : an
kelang : an
itim : in
tuwa : ng
tau : ng
taun : g
annoy : ing
annoyi : ng
annoyin : g
ano : ng
andam : ing
andami : ng
syado : ng
syadon : g
sobra : ng
givi : ng
givin : g
earn : ing
earni : ng
earnin : g
syado : ng
syadon : g
halata : ng
daa : ng
daan : g
risi : ng
risin : g
kair : ita
nothi : ng
nothin : g
gawi : ng
igi : ng
suporta 

IndexError: string index out of range